In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import numpy as np
from qiskit import QuantumCircuit
import Dependencies.functions_list as df
import Dependencies.functions_listCpp as dfcpp

ModuleNotFoundError: No module named 'Dependencies.functions_listCpp'

In [ ]:
n = 3
qc = QuantumCircuit(n)
qc.h([0,1,2])
qc.cz(0,1)
qc.s(1)
qc.h(1)
# qc.s(0)
qc.t(0)
qc.z(1)
qc.ccz(0,1,2)
d = qc.depth()
print(qc)

     ┌───┐   ┌───┐             
q_0: ┤ H ├─■─┤ T ├───────────■─
     ├───┤ │ ├───┤┌───┐┌───┐ │ 
q_1: ┤ H ├─■─┤ S ├┤ H ├┤ Z ├─■─
     ├───┤   └───┘└───┘└───┘ │ 
q_2: ┤ H ├───────────────────■─
     └───┘                     


In [ ]:
instructions = [(instruction.operation.name,
                    [qc.find_bit(q).index for q in instruction.qubits]) 
                    for index, instruction in enumerate(qc.data)]
print(instructions)

[('h', [0]), ('h', [1]), ('h', [2]), ('cz', [0, 1]), ('s', [1]), ('h', [1]), ('t', [0]), ('z', [1]), ('ccz', [0, 1, 2])]


In [ ]:
terms, wire_array, max_new_var = df.create_poly(qc, n)
print(terms)
print(wire_array)
print(max_new_var)
ovs = [sub[-1] for sub in wire_array]
print(ovs)

Initial wire_array:  [[0], [1], [2]]
[[4, [0, 3]], [4, [1, 4]], [4, [2, 5]], [4, [3, 4]], [2, [4]], [4, [4, 6]], [1, [3]], [4, [6]], [4, [3, 6, 5]]]
[[0, 3], [1, 4, 6], [2, 5]]
7
[3, 6, 5]


In [ ]:
initial_state = np.zeros(n, dtype=bool)
print(initial_state)
tt = df.get_truthtable(terms, n, max_new_var, initial_state)
print(tt)

[False False False]
16
[0 4 0 4 2 2 2 2 1 5 1 1 7 7 7 3]


In [ ]:
statevector = df.get_statevector(tt, n, max_new_var, ovs)
print(type(statevector))
print("Statevector using BOOLEAN equation: ")
threshold = 1e-8
for i, amp in enumerate(statevector):
    if abs(amp) > threshold:
        print(f"|{i:0{n}b}> : {amp:.6f}")

{0: array([1, 0, 1, 0, 0, 0, 0, 0]), 2: array([0, 0, 1, 0, 1, 0, 0, 0]), 1: array([1, 0, 1, 0, 0, 0, 0, 0]), 3: array([0, 0, 1, 0, 1, 0, 0, 0]), 4: array([0, 1, 0, 0, 0, 0, 0, 1]), 6: array([0, 0, 0, 0, 0, 1, 0, 1]), 5: array([0, 1, 0, 0, 0, 0, 0, 1]), 7: array([0, 1, 0, 1, 0, 0, 0, 0])}
<class 'numpy.ndarray'>
Statevector using BOOLEAN equation: 
|000> : 0.250000+0.250000j
|001> : 0.250000+0.250000j
|010> : -0.250000+0.250000j
|011> : -0.250000+0.250000j
|100> : 0.353553+0.000000j
|101> : 0.353553+0.000000j
|110> : 0.000000-0.353553j
|111> : 0.000000+0.353553j
